In [1]:
import requests
from queue import Queue
from threading import Thread
import time

# API endpoint
API_URL = "https://assetwise.co.th/api/"

# Number of workers (threads)
NUM_WORKERS = 100

# Function for each worker to process tasks from the queue
def worker(task_queue, results):
    while not task_queue.empty():
        try:
            # Get a task from the queue
            _ = task_queue.get_nowait()
            # Send the request
            response = requests.get(API_URL, timeout=5)
            # Record the status code or error
            results.append(response.status_code if response.status_code == 200 else f"Error: {response.status_code}")
        except requests.exceptions.RequestException as e:
            results.append(f"Error: {e}")
        finally:
            task_queue.task_done()

# Main function to manage the worker pool
def send_requests_with_workers(total_requests, num_workers):
    start_time = time.time()

    # Create a task queue and fill it with tasks
    task_queue = Queue()
    for _ in range(total_requests):
        task_queue.put(None)  # Task doesn't need data in this case

    # List to collect results
    results = []

    # Create and start the workers
    threads = []
    for _ in range(num_workers):
        thread = Thread(target=worker, args=(task_queue, results))
        thread.start()
        threads.append(thread)

    # Wait for all tasks to complete
    task_queue.join()

    # Wait for all threads to finish
    for thread in threads:
        thread.join()

    end_time = time.time()

    # Analyze the results
    success_count = sum(1 for result in results if result == 200)
    fail_count = len(results) - success_count

    print(f"Total Requests Sent: {total_requests}")
    print(f"Successful Requests: {success_count}")
    print(f"Failed Requests: {fail_count}")
    print(f"Time Taken: {end_time - start_time:.2f} seconds")

if __name__ == "__main__":
    # Set the total number of requests
    TOTAL_REQUESTS = 100000  # Total number of requests to send

    # Call the function with NUM_WORKERS
    send_requests_with_workers(TOTAL_REQUESTS, NUM_WORKERS)

KeyboardInterrupt: 